#### Import necessary libraries

In [ ]:
import botocore
import boto3
import pprint

from langchain_aws import ChatBedrock
from langchain_aws import BedrockEmbeddings
from langchain_community.retrievers import AmazonKnowledgeBasesRetriever

from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

#### Verify the ID for the existing Knowledge Base in Amazon Bedrock:

In [2]:
session = boto3.Session()
bedrock_client = session.client('bedrock-agent')

try:
    response = bedrock_client.list_knowledge_bases(
        maxResults=1  
    )
    knowledge_base_summaries = response.get('knowledgeBaseSummaries', [])

    if knowledge_base_summaries:
        kb_id = knowledge_base_summaries[0]['knowledgeBaseId']
        print(f"Knowledge Base ID: {kb_id}")
    else:
        print("No Knowledge Base summaries found.")
        
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

No Knowledge Base summaries found.


#### Select the models that will be consumed by the knowledgebase

In [3]:
pp = pprint.PrettyPrinter(indent=2)

bedrock_client = boto3.client('bedrock-runtime')

llm_for_text_generation = ChatBedrock(model_id="amazon.nova-lite-v1:0", client=bedrock_client)
llm_for_evaluation = ChatBedrock(model_id="amazon.nova-lite-v1:0", client=bedrock_client)

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0",client=bedrock_client)

#### Create a *AmazonKnowledgeBasesRetriever* object using langchain

In [12]:
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 5}},
        region_name="us-east-1",
    )
retriever

4. Run the following code cell to create a prompt with context and question as variables:

In [ ]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""
prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context","question"])

5. Run the following code cell to invoke the model using a pre-defined query and print the results:

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs): #concatenate the text from the page_content field in the output from retriever.invoke
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm_for_text_generation
    | StrOutputParser()
)

query = "Provide a list of ten risks for AnyCompany financial as a numbered list. Do not include descriptions."

response=chain.invoke(query)
print(response)

#### Prepare the *question* and *ground_truths* pairs for evaluation:

In [14]:
# --- Prompt Template for Guardrails and Persona ---
prompt_template_string = """
You are an internal AI assistant for a multi-state outpatient psychiatric group. Your primary purpose is to provide accurate information from the provided Standard Operating Procedures (SOPs), policies, and internal documentation.

**Strictly adhere to the following rules:**
1.  **ONLY** answer questions using the information provided in the given context.
2.  If the answer to the question is not explicitly found in the provided context, state clearly: 'I cannot find an answer to that question in the provided documents.' **DO NOT** use your general knowledge or make up information.
3.  For every piece of information you provide, cite the document name and relevant section/page number from the source.
4.  **DO NOT** provide any medical advice, diagnoses, or treatment recommendations.
5.  **DO NOT** discuss any topics unrelated to the clinic's internal operations, policies, or SOPs. If asked an unrelated question, respond with: 'My purpose is to provide information from our internal clinic documents. I am unable to answer questions outside this scope.'
6.  Maintain a professional and helpful tone.

Context: {context}

Question: {question}

Answer:
"""
PROMPT = PromptTemplate(
    template=prompt_template_string,
    input_variables=["context", "question"]
)

# --- Create the RAG Chain using RetrievalQA ---
chain = RetrievalQA.from_chain_type(
    llm=llm_for_text_generation,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT} 
)

# --- The test data ---
questions = [
    "Why are Standard Operating Procedures (SOPs) considered essential for HIPAA compliant communication?",
    "What are the recommended steps for developing an effective HIPAA compliant communication SOP?",
    "What are the primary goals of the AHRQ Surveys on Patient Safety Culture (SOPS) program?",
    "In which healthcare settings are the AHRQ SOPS surveys designed to assess patient safety culture?",
    "What is the main purpose of the American Psychiatric Association (APA) Clinical Practice Guidelines?",
    "Name at least three psychiatric disorders for which the APA provides specific clinical practice guidelines.",
    "According to the APA's sample privacy policies, what is required to be provided to patients regarding the practice's use of Protected Health Information (PHI) at their first office visit?",
    "What does the HIPAA Privacy Rule establish national standards for, as detailed in the APA's checklist?",
    "Can a healthcare provider leave Protected Health Information (PHI) in a voicemail message according to HIPAA compliant communication SOPs?",
    "What should be done when a new communication technology is introduced in a healthcare setting concerning HIPAA compliance?"
]
ground_truth = [
    "SOPs are essential for HIPAA compliant communication because they provide clear guidelines for handling protected health information (PHI), ensure consistency in how employees handle PHI (reducing human errors that could lead to violations), and serve as a reference for audits and compliance reviews.",
    "Developing an effective HIPAA compliant communication SOP involves several steps: identifying communication channels, defining security protocols for each channel, implementing access controls and authentication, training employees on HIPAA compliance, developing an incident response plan, and regularly reviewing and updating SOPs.",
    "The primary goals of the AHRQ Surveys on Patient Safety Culture (SOPS) program are to advance the scientific understanding of patient safety culture in healthcare settings, conduct activities to understand, measure, and improve patient safety culture, and support healthcare organizations in assessing and improving their patient safety culture.",
    "The AHRQ SOPS surveys are designed to assess patient safety culture in various healthcare settings including hospitals, medical offices, nursing homes, community pharmacies, and ambulatory surgery centers.",
    "The main purpose of the American Psychiatric Association (APA) Clinical Practice Guidelines is to provide evidence-based recommendations for the assessment and treatment of psychiatric disorders, assisting in clinical decision-making by presenting systematically developed patient care strategies in a standardized format.",
    "The APA provides specific clinical practice guidelines for disorders such as Schizophrenia, Major Depressive Disorder (Joint VA/DoD Guideline), and Posttraumatic Stress Disorder and Acute Stress Reaction (Joint VA/DoD Guideline). Other related guidelines include those for Alcohol Withdrawal Management and the Health of Transgender and Gender Diverse People.",
    "According to the APA's sample privacy policies, healthcare providers are required to furnish patients with a written Notice of Privacy Practices at the time of their first office visit. This notice describes how the practice handles confidential patient information in accordance with HIPAA regulations.",
    "The HIPAA Privacy Rule, as detailed in the APA's checklist, establishes national standards to protect individuals' medical records and other individually identifiable health information (PHI) in any format it is created, received, maintained, or transmitted.",
    "According to HIPAA compliant communication SOPs, healthcare providers should not leave Protected Health Information (PHI) in voicemail messages. Instead, they should request a callback.",
    "When a new communication technology is introduced in a healthcare setting, HIPAA compliant SOPs should be immediately developed or adapted for that channel. This ensures that the new technology meets all security, privacy, and compliance standards before Protected Health Information (PHI) is transmitted through it."
]

answers = []
contexts = [] 

for query in questions:
    # RetrievalQA.invoke expects a dictionary with a 'query' key
    response = chain.invoke({"query": query})
    
    # The generated answer is in the 'result' key of the response
    answers.append(response["result"])
    
    # The source documents are in the 'source_documents' key
    # Extract page_content from each document
    contexts.append([doc.page_content for doc in response.get("source_documents", [])])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truth
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

print("\n--- Evaluation Dataset Generated ---")
print(dataset[0]) # Print the first entry to verify structure


--- Evaluation Dataset Generated ---
{'question': 'Why are Standard Operating Procedures (SOPs) considered essential for HIPAA compliant communication?', 'answer': 'Standard Operating Procedures (SOPs) are considered essential for HIPAA compliant communication because they ensure consistency in how employees handle Protected Health Information (PHI), reducing human errors that could lead to violations. Well-documented procedures also serve as a reference for audits and compliance reviews. \n\n**Source:**\n- Document: "SOPs for HIPAA compliant communication" \n- Section: "Why SOPs are essential for HIPAA compliance"', 'contexts': ['SOPs for HIPAA compliant communication 2 min read # SOPs for HIPAA compliant communication ![Picture of Tshedimoso Makhene](https://app.hubspot.com/settings/avatar/d41d8cd98f00b204e9800998ecf8427e) Tshedimoso Makhene Feb 11, 2025 2:39:17 PM HIPAA Compliance SOPs for HIPAA compliant communication Ensuring HIPAA compliance in communication is a critical respon

#### See the answers from the LLM and the ground truths for the evaluation set of questions:

In [15]:
i=0
for answer in answers:
    i=i+1
    print(str(i)+').'+questions[i-1]+'\n')
    print("LLM:" +answer+'\n')
    print ("Ground truth: "+ ground_truth[i-1]+'\n')

1).Why are Standard Operating Procedures (SOPs) considered essential for HIPAA compliant communication?

LLM:Standard Operating Procedures (SOPs) are considered essential for HIPAA compliant communication because they ensure consistency in how employees handle Protected Health Information (PHI), reducing human errors that could lead to violations. Well-documented procedures also serve as a reference for audits and compliance reviews. 

**Source:**
- Document: "SOPs for HIPAA compliant communication" 
- Section: "Why SOPs are essential for HIPAA compliance"

Ground truth: SOPs are essential for HIPAA compliant communication because they provide clear guidelines for handling protected health information (PHI), ensure consistency in how employees handle PHI (reducing human errors that could lead to violations), and serve as a reference for audits and compliance reviews.

2).What are the recommended steps for developing an effective HIPAA compliant communication SOP?

LLM:The recommended s

#### Import all the metrics from *ragas.metrics* and use the *evaluate()* function:

In [16]:
import warnings
warnings.filterwarnings('ignore')  

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_similarity,
    answer_correctness
)

from ragas.metrics.critique import (
harmfulness, 
maliciousness, 
coherence, 
correctness, 
conciseness
)

#specify the metrics here
metrics = [
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
        context_entity_recall,
        answer_similarity,
        answer_correctness,
        harmfulness, 
        maliciousness, 
        coherence, 
        correctness, 
        conciseness
    ]

result = evaluate(
    dataset = dataset, 
    metrics=metrics,
    llm=llm_for_evaluation,
    embeddings=bedrock_embeddings,
)

df = result.to_pandas()
result

Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

#### Run the below code cell to see the resulting detailed RAGAS scores:

In [17]:
import pandas as pd
pd.options.display.max_colwidth = 10
df.style.set_sticky(axis="columns")

#### Export the resulting RAGAS scores in excel format:

In [18]:
df.style.to_excel('styled.xlsx', engine='openpyxl')